In [ ]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_moons(n_samples = 500, noise = 0.3, random_state = 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

"""
VotingClassifier는 여러 개의 모델을 estimators에 넣고 각자 모델을 훈련하여
각각의 모델이 각자의 예측을 반환하고 그 예측의 과반수를 선택하는 분류기다.
앙상블 모델은 앙상블을 이루는 모델이 독립적일수록 매우 다른 종류의 오차를
반환하므로 일반적으로 성능이 향상된다.
"""
voting_clf = VotingClassifier(
    estimators = [
        ('lr', LogisticRegression(random_state = 42)),
        ('rf', RandomForestClassifier(random_state = 42)),
        ('svc', SVC(random_state = 42))
    ]
)

voting_clf.fit(X_train, y_train)

In [ ]:
voting_clf.score(X_test, y_test)

0.912

In [ ]:
"""모든 분류기가 다중 분류를 지원하면 (predict_proba 메서드가 있으면)
간접 투표 방식을 시행할 수 있다. 이는 위의 단순한 방법보다 더욱 성능이 좋다.
"""

voting_clf.voting = 'soft'
voting_clf.named_estimators["svc"].probability = True # 이렇게 직접 수정도 가능
voting_clf.fit(X_train, y_train)
voting_clf.score(X_test, y_test)

0.92

In [ ]:
"""
Bagging (Bootstrap aggregationg) : 훈련 세트에서 중복을 허용하여 샘플링(뽑고 다시 넣고 뽑고)
Pasting : 중복을 허용하지 않음

배깅을 사용하면 하나의 알고리즘을 사용한 여러개의 분류기를 각기 다르게 훈련하여
분류의 경우에는(최빈 예측값) 회귀의 경우에는 평균값을 계산해 더욱 좋은 결과를 얻을 수 있다.
배깅은 편향을 손해보고 분산을 줄이는 방식이로 전체적으로 더 일반화된 모델을 만든다.
"""
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 결정트리 500개를 배깅 앙상블로 만드는 것.
# bootstrap = False로 지정하면 페이스팅으로 훈련함.
# oob_score = True로 지정하면 Out of Bag 샘플을 가지고 검증세트로 활용
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 500,
                            max_samples = 100, n_jobs = -1, random_state = 42,
                            bootstrap = True, oob_score = True)
bag_clf.fit(X_train, y_train)
print(f"예측 점수 :{bag_clf.score(X_test, y_test)}")
print(f"OOB 점수 : {bag_clf.oob_score_.round(3)}")

y_pred = bag_clf.predict(X_test)
print(f"테스트 세트 정확도: {accuracy_score(y_test, y_pred)}")



예측 점수 :0.904
OOB 점수 : 0.925
테스트 세트 정확도: 0.904


In [ ]:
#랜덤포레스트 모델은 기본적으로 샘플과 특성에 배깅(또는 페이스팅)을 적용한다.
from sklearn.ensemble import RandomForestClassifier

#샘플 개수를 지정하지 않으면 전체를 배깅함
#특성의 경우는 sqrt(n)개를 골라서 무작위성을 주입함.
rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16,
                                 n_jobs = -1, random_state = 42)
rnd_clf.fit(X_train, y_train)

#랜덤 포레스트는 각 특성의 중요도를 반환한다.
rnd_clf.feature_importances_

"""
일반적인 랜덤 포레스트보다 더 극단적으로 랜덤(Extremely Random)하고
더욱 빠른 트리 알고리즘을 Extra Tree라고 한다.
ensemble.ExtraTreesClassifier()
기본적으로 모든 샘플을 사용한다. bootstrap = False
"""


In [ ]:
"""
Boosting은 약한 학습기를 연결해서 강한 학습기를 만드는 앙상블 기법으로
앞의 모델의 가중치(AdaBoost)나 잔여 오차(GBRT)를 기반으로 다음 모델을
지속적으로 훈련시켜 더욱 강력한 모델을 만드는 방법을 말한다.

n_estimator = 연결 시킬 모델의 개수
learning_rate = 각 모델(트리)의 기여도, 낮게 설정할수록 많은 트리가 필요하지만
                일반적으로 예측의 성능은 향상된다.
"""
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 1), n_estimators = 30,
    learning_rate = 0.5, random_state = 42)

gbrt = GradientBoostingClassifier(max_depth = 2, n_estimaotr = 5,
                                  learning_rate = 1.0, random_state = 42)
"""
gbrt의 n_iter_no_change = 10처럼 설정하면 early_stopping을 구현할 수 있다.
n_iter_no_change를 설정하면 fit메서드가 자동으로 훈련세트를 작은 훈련세트와
검증 세트로 분할하며. 이 비율은 validation_fraction = 0.1(10%)처럼 지정할 수 있다.
또한 tol = 1e-4(0.0001)은 무시할 수 있는 최대 성능 향상값을 결정한다.

subsample = 0.25는 각 트리가 훈련할 때 사용할 훈련 샘플의 비율이다.
각 트리 훈련이 시작할때마다 25%씩 랜덤으로 골라서 훈련하며
이는 편향이 높아지는 대신 분산이 낮아지고, 훈련 속도 또한 빨라지는 효과를 가진다.
(Stochastic gradient boosting)
"""


In [ ]:
"""
추가로 sample이 10000개 이상이 되면 HistGradientBoosting이 더욱 효과적인데
계산복잡도가 O(b*m) b = 구간의 개수(max = 255), m = sample의 개수 이므로
엄청나게 빠른 계산을 구현한다. 다만 구간 분할 자체가 규제처럼 작동해
정밀도 손실을 유발하므로, 과대적합이나 과소적합의 원인이 될 수 있다.

특징:
1. 전처리 필요 없음(트리가 그렇듯)
2. 특성을 신경 쓸 필요 없음(특성 정렬 불필요)
3. 훈련 속도가 매우 빠름(O(b*m))
4. 범주형 특성과 누락된 값 지원

일반 그레이디언트 부스팅과의 차이점:
1. subsample 지원하지 않음
2. n_estimator가 max_iter로 바뀜
3. max_leaf_nodes, min_samples_leaf, max_depth, max_bins정도밖에 설정 불가능


아래는 예시
1. 범주형 특성을 0~ max_bins사이의 정수로 표현하기 위해 OrdinalEncoder 사용
2. 범주형 열의 인덱스로 categorical_features = [n] 을 설정해야함.

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder

hgb_reg = make_pipeline(
    make_column_transformer((OrdinalEncoder(), ["ocean_proximity"]),
                            remainder="passthrough"),
    HistGradientBoostingRegressor(categorical_features=[0], random_state=42)
)
hgb_reg.fit(housing, housing_labels)
"""

In [ ]:
"""
Stacking (Stacked generalization)은 각각의 모델의 예측을 하나로 합친
블렌딩 훈련 세트를 마지막 예측기(Blender or Meta learner)의 입력으로 사용해
마지막 예측기를 훈련시켜 최종 예측을 만드는 앙상블 기법이다.
"""

from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    final_estimator=RandomForestClassifier(random_state=43),
    cv=5  # 교차 검증 폴드 수
)
stacking_clf.fit(X_train, y_train)
stacking_clf.score(X_test, y_test)

0.928